In [1]:
from keras import models, layers, optimizers
from keras.preprocessing import sequence
from keras.datasets import imdb

In [41]:
max_features = 10000
maxlen = 500
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)
print(len(x_train))
print(len(x_test))

25000
25000


In [42]:
x_train = sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = sequence.pad_sequences(x_test, maxlen=maxlen)
print(x_train.shape)
print(x_test.shape)

(25000, 500)
(25000, 500)


In [43]:
model = models.Sequential()
model.add(layers.Embedding(max_features, 128, input_length=max_len))
model.add(layers.Conv1D(32, 7, activation='relu'))
model.add(layers.MaxPool1D(5))
model.add(layers.Conv1D(32, 7, activation='relu'))
model.add(layers.GlobalMaxPool1D())
model.add(layers.Dense(1))
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 500, 128)          1280000   
_________________________________________________________________
conv1d_11 (Conv1D)           (None, 494, 32)           28704     
_________________________________________________________________
max_pooling1d_7 (MaxPooling1 (None, 98, 32)            0         
_________________________________________________________________
conv1d_12 (Conv1D)           (None, 92, 32)            7200      
_________________________________________________________________
global_max_pooling1d_4 (Glob (None, 32)                0         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 33        
Total params: 1,315,937
Trainable params: 1,315,937
Non-trainable params: 0
_________________________________________________________________


In [44]:
model.compile(optimizer=optimizers.RMSprop(lr=1e-4),
              loss='binary_crossentropy',
              metrics=['acc'])

history = model.fit(x_train,
                    y_train,
                    epochs=10,
                    batch_size=128,
                    validation_split=0.2)

Train on 20000 samples, validate on 5000 samples
Epoch 1/10
20000/20000 [==============================] - 52s 3ms/step - loss: 0.8336 - acc: 0.5092 - val_loss: 0.6874 - val_acc: 0.5646
Epoch 2/10
20000/20000 [==============================] - 52s 3ms/step - loss: 0.6699 - acc: 0.6393 - val_loss: 0.6641 - val_acc: 0.6586
Epoch 3/10
20000/20000 [==============================] - 51s 3ms/step - loss: 0.6234 - acc: 0.7530 - val_loss: 0.6078 - val_acc: 0.7440
Epoch 4/10
20000/20000 [==============================] - 52s 3ms/step - loss: 0.5255 - acc: 0.8075 - val_loss: 0.4844 - val_acc: 0.8052
Epoch 5/10
20000/20000 [==============================] - 49s 2ms/step - loss: 0.4107 - acc: 0.8491 - val_loss: 0.4292 - val_acc: 0.8306
Epoch 6/10
20000/20000 [==============================] - 49s 2ms/step - loss: 0.3520 - acc: 0.8710 - val_loss: 0.4116 - val_acc: 0.8390
Epoch 7/10
20000/20000 [==============================] - 50s 3ms/step - loss: 0.3139 - acc: 0.8739 - val_loss: 0.4380 - val_acc:

convnet for tempurature

In [16]:
import os
import numpy as np

In [17]:
data_dir = 'F:\study\ml\DataSet\jena_climate'
fname = os.path.join(data_dir, 'jena_climate_2009_2016.csv')

f = open(fname)
data = f.read()
f.close()

In [18]:
lines = data.split('\n')
header = lines[0].split(',')
lines = lines[1:]
print(header)
print(len(lines))

['"Date Time"', '"p (mbar)"', '"T (degC)"', '"Tpot (K)"', '"Tdew (degC)"', '"rh (%)"', '"VPmax (mbar)"', '"VPact (mbar)"', '"VPdef (mbar)"', '"sh (g/kg)"', '"H2OC (mmol/mol)"', '"rho (g/m**3)"', '"wv (m/s)"', '"max. wv (m/s)"', '"wd (deg)"']
420551


In [19]:
float_data = np.zeros((len(lines), len(header) - 1))
for i, line in enumerate(lines):
    values = [float(x) for x in line.split(',')[1:]]
    float_data[i, :] = values

In [21]:
mean = float_data[:20000].mean(axis=0)
float_data -= mean
std = float_data[:20000].std(axis=0)
float_data /= std

In [24]:
def generator(data,
              lookback,
              delay,
              min_index,
              max_index,
              shuffle=False,
              batch_size=128,
              step=6):
    if max_index is None:
        max_index = len(data) - delay - 1
    i = min_index + lookback

    while 1:
        if shuffle:
            rows = np.random.randint(min_index + lookback, max_index,
                                     batch_size)
        else:
            if i + batch_size > max_index:
                i = min_index + lookback
            rows = np.arange(i, min(i + batch_size, max_index))
            i += len(rows)
#         print(rows)
#         print(len(rows))

        samples = np.zeros((len(rows), lookback // step, data.shape[-1]))
        targets = np.zeros((len(rows), ))
        #         print(samples.shape)

        for j, row in enumerate(rows):
            indices = range(rows[j] - lookback, rows[j], step)
            #             print(indices)
            samples[j] = data[indices]
            targets[j] = data[rows[j] + delay][1]


#             print(samples.shape)
        yield samples, targets

In [25]:
lookback = 1440
step = 6
delay = 144
batch_size = 128

In [26]:
train_gen = generator(float_data,
                      lookback=lookback,
                      delay=delay,
                      min_index=0,
                      max_index=200000,
                      shuffle=True,
                      step=step,
                      batch_size=batch_size)
val_gen = generator(
    float_data,
    lookback=lookback,
    delay=delay,
    min_index=200001,
    max_index=300000,
    #                       shuffle=True,
    step=step,
    batch_size=batch_size)

In [28]:
val_steps = (300000 - 200001 - lookback) // batch_size

In [23]:
model2 = models.Sequential()
model2.add(
    layers.Conv1D(32,
                  5,
                  activation='relu',
                  input_shape=(None, float_data.shape[-1])))
model2.add(layers.MaxPooling1D(3))
model2.add(layers.Conv1D(32, 5, activation='relu'))
model2.add(layers.MaxPooling1D(3))
model2.add(layers.Conv1D(32, 5, activation='relu'))
model2.add(layers.GlobalMaxPooling1D())
model2.add(layers.Dense(1))

In [29]:
model2.compile(optimizer=optimizers.RMSprop(), loss='mae')

history2 = model2.fit_generator(train_gen,
                                steps_per_epoch=500,
                                epochs=20,
                                validation_data=val_gen,
                                validation_steps=val_steps)

Epoch 1/20
500/500 [==============================] - 33s 66ms/step - loss: 0.4844 - val_loss: 0.5199
Epoch 2/20
500/500 [==============================] - 32s 65ms/step - loss: 0.4213 - val_loss: 0.5033
Epoch 3/20
500/500 [==============================] - 33s 65ms/step - loss: 0.3936 - val_loss: 0.4993
Epoch 4/20
500/500 [==============================] - 33s 65ms/step - loss: 0.3753 - val_loss: 0.5046
Epoch 5/20
500/500 [==============================] - 32s 65ms/step - loss: 0.3608 - val_loss: 0.5633
Epoch 6/20
500/500 [==============================] - 32s 65ms/step - loss: 0.3524 - val_loss: 0.5576
Epoch 7/20
500/500 [==============================] - 32s 65ms/step - loss: 0.3435 - val_loss: 0.5211
Epoch 8/20
500/500 [==============================] - 32s 65ms/step - loss: 0.3374 - val_loss: 0.5181
Epoch 9/20
500/500 [==============================] - 32s 65ms/step - loss: 0.3304 - val_loss: 0.5261
Epoch 10/20
500/500 [==============================] - 33s 65ms/step - loss: 0.324

In [30]:
step = 3
lookback = 720
delay = 144

In [32]:
train_gen = generator(float_data,
                      lookback=lookback,
                      delay=delay,
                      min_index=0,
                      max_index=200000,
                      shuffle=True,
                      step=step,
                      batch_size=batch_size)
val_gen = generator(
    float_data,
    lookback=lookback,
    delay=delay,
    min_index=200001,
    max_index=300000,
    #                       shuffle=True,
    step=step,
    batch_size=batch_size)

In [31]:
val_steps = (300000 - 200001 - lookback) // batch_size

In [33]:
float_data.shape

(420551, 14)

In [37]:
model3=models.Sequential()
model3.add(layers.Conv1D(32,5,activation='relu',
                        input_shape=(None,float_data.shape[-1])))
model3.add(layers.MaxPooling1D(3))
model3.add(layers.Conv1D(32,5,activation='relu'))
model3.add(layers.GRU(32,dropout=0.1,recurrent_dropout=0.5))
model3.add(layers.Dense(1))
model3.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_9 (Conv1D)            (None, None, 32)          2272      
_________________________________________________________________
max_pooling1d_6 (MaxPooling1 (None, None, 32)          0         
_________________________________________________________________
conv1d_10 (Conv1D)           (None, None, 32)          5152      
_________________________________________________________________
gru_1 (GRU)                  (None, 32)                6240      
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 33        
Total params: 13,697
Trainable params: 13,697
Non-trainable params: 0
_________________________________________________________________


In [40]:
model3.compile(optimizer=optimizers.RMSprop(),loss='mae')
history3=model3.fit_generator(train_gen,
                             steps_per_epoch=500,
                             epochs=20,
                             validation_data=val_gen,
                             validation_steps=val_steps)

Epoch 1/20
500/500 [==============================] - 55s 109ms/step - loss: 0.3873 - val_loss: 0.3257
Epoch 2/20
500/500 [==============================] - 56s 111ms/step - loss: 0.3472 - val_loss: 0.3163
Epoch 3/20
500/500 [==============================] - 57s 114ms/step - loss: 0.3369 - val_loss: 0.3175
Epoch 4/20
500/500 [==============================] - 75s 151ms/step - loss: 0.3274 - val_loss: 0.3138
Epoch 5/20
500/500 [==============================] - 58s 116ms/step - loss: 0.3178 - val_loss: 0.3237
Epoch 6/20
500/500 [==============================] - 62s 124ms/step - loss: 0.3125 - val_loss: 0.3179
Epoch 7/20
500/500 [==============================] - 60s 121ms/step - loss: 0.3056 - val_loss: 0.3218
Epoch 8/20
500/500 [==============================] - 61s 122ms/step - loss: 0.3007 - val_loss: 0.3204
Epoch 9/20
500/500 [==============================] - 62s 124ms/step - loss: 0.2961 - val_loss: 0.3253
Epoch 10/20
500/500 [==============================] - 57s 114ms/step - l